In [2]:
import pandas as pd
import numpy as np
import os
import json
import altair as alt
import numpy as np

In [3]:
#JSON_FILE_BUSTEDS_MH = "../analysis/13-datasets/BUSTEDS-MH/yokoyama.rh1.cds.mod.1-990.nex.BUSTEDS-MH.json"
#JSON_FILE_BUSTEDS = "../analysis/13-datasets/BUSTEDS/yokoyama.rh1.cds.mod.1-990.nex.BUSTEDS.json"
#JSON_FILE_MH = "../data/13-datasets/yokoyama.rh1.cds.mod.1-990.nex.FITTER.json"

#JSON_FILE_BUSTEDS_MH = "../analysis/13-datasets/BUSTEDS-MH/HepatitisD.nex.BUSTEDS-MH.json"
#JSON_FILE_BUSTEDS = "../analysis/13-datasets/BUSTEDS/HepatitisD.nex.BUSTEDS.json"
#JSON_FILE_MH = "../data/13-datasets/HepatitisD.nex.FITTER.json"

pvalueThreshold_MH = 0.05
pvalueThreshold_BUSTEDS_MH = 0.1
ER_THRESHOLD = 5

In [10]:
# MAC OSX
WD = "/Users/user/Documents/BUSTEDS-MH"

BUSTEDS_DIR = os.path.join(WD, "analysis/13-datasets/BUSTEDS")
BUSTEDS_MH_DIR = os.path.join(WD, "analysis/13-datasets/BUSTEDS-MH")

BUSTEDS_DIR_FILES = [os.path.join(BUSTEDS_DIR, file.name) for file in os.scandir(BUSTEDS_DIR) if file.name.endswith(".json")]
BUSTEDS_MH_DIR_FILES = [os.path.join(BUSTEDS_MH_DIR, file.name) for file in os.scandir(BUSTEDS_MH_DIR) if file.name.endswith(".json")]

print("# Number of BUSTEDS results:", len(BUSTEDS_DIR_FILES))
print("# Number of BUSTEDS-MH results:", len(BUSTEDS_MH_DIR_FILES))



# Number of BUSTEDS results: 13
# Number of BUSTEDS-MH results: 13


In [4]:
def read_json(filename):
    print("# Reading:", filename)
    if os.stat(filename).st_size == 0: 
        return []
    #end if
    with open(filename, "r") as fh:
        json_data = json.load(fh)
    fh.close()
    return json_data
#end method

## Transform the data, easier to plot

In [53]:
def create_df(x, y):
    try:
        num_sites = len(x["Evidence Ratios"]["constrained"][0])
        print("# Number of sites (BUSTEDS-MH):", num_sites)
    except:
        return 1
    pass

    res = [x+1 for x in range(num_sites)]

    # BUSTEDS-MH
    bs = ["BUSTEDS-MH"] * num_sites
    data_tuples = list(zip(res, x["Evidence Ratios"]["constrained"][0], bs))
    # Site, ER_Value, Model
    df1 = pd.DataFrame(data_tuples, columns=['Site','Evidence Ratio', "Method"])

    # BUSTEDS
    bs = ["BUSTEDS"] * num_sites
    data_tuples = list(zip(res, y["Evidence Ratios"]["constrained"][0], bs))
    # Site, ER_Value, Model
    df3 = pd.DataFrame(data_tuples, columns=['Site','Evidence Ratio', "Method"])
    
    #result = pd.concat(df_list)
    result = pd.concat([df1, df3])

    return result


def create_df_log10(x, y):
    try:
        num_sites = len(x["Evidence Ratios"]["constrained"][0])
        print("# Number of sites (BUSTEDS-MH):", num_sites)
    except:
        return 1
    pass

    res = [x+1 for x in range(num_sites)]

    # BUSTEDS-MH
    bs = ["BUSTEDS-MH"] * num_sites
    data_tuples = list(zip(res, np.log10(x["Evidence Ratios"]["constrained"][0]), bs))
    
    # Site, ER_Value, Model
    df1 = pd.DataFrame(data_tuples, columns=['Site','Evidence Ratio', "Method"])

    # BUSTEDS
    bs = ["BUSTEDS"] * num_sites
    data_tuples = list(zip(res, np.log10(y["Evidence Ratios"]["constrained"][0]), bs))
    
    # Site, ER_Value, Model
    df3 = pd.DataFrame(data_tuples, columns=['Site','Evidence Ratio', "Method"])
    
    result = pd.concat([df1, df3])

    return result

## Pure ER's between MH, BUSTEDS, and BUSTEDS-MH

## Visualize

In [70]:
def plot(result, JSON_FILE_BUSTEDS_MH, title_text, outfile,clamp=False):
    source = result
    #title_text = os.path.basename(JSON_FILE_BUSTEDS_MH).replace(".BUSTEDS-MH.json", "")
    
    print("# Plotting:", title_text)
    
    if clamp == False:
        line = alt.Chart(source, title=title_text).mark_area().encode(
            x='Site',
            y='sum(Evidence Ratio)',
            color='Method'
        ).properties(
            width=1200,
            height=800,
            title=title_text).configure_axis(
        labelFontSize=18,
        titleFontSize=18).configure_title(fontSize=24)
        #line
        #chart.save('chart.png')
        #line.save(title_text + ".png")
        line.save(outfile)
        
    if clamp == True:
        line = alt.Chart(source, title=title_text).mark_area().encode(
            x='Site',
            y=alt.Y('sum(Evidence Ratio)', scale=alt.Scale(domain=(-2, 10), clamp=True)),
            color='Method'
        ).properties(
            width=1200,
            height=800,
            title=title_text).configure_axis(
        labelFontSize=18,
        titleFontSize=18).configure_title(fontSize=24)
        
        line.save(outfile)
        
#plot(result)

## Main

In [71]:
def main(JSON_FILE_BUSTEDS_MH, JSON_FILE_BUSTEDS):
    
    title_text = os.path.basename(JSON_FILE_BUSTEDS_MH).replace(".BUSTEDS-MH.json", "")
    
    x = read_json(JSON_FILE_BUSTEDS_MH)
    y = read_json(JSON_FILE_BUSTEDS)
    
    result = create_df(x, y)
    result_log10 = create_df_log10(x, y)
    
    if type(result) != pd.core.frame.DataFrame:
        print("# No ER data")
        print()
        return # Error
    else:
        #print(result)
        outfile = title_text + ".png"
        plot(result, JSON_FILE_BUSTEDS_MH, title_text, outfile)
        outfile_log10 = title_text + "_log10.png"
        plot(result_log10, JSON_FILE_BUSTEDS_MH, title_text, outfile_log10)
        
        outfile_log10 = title_text + "_log10_clamp.png"
        plot(result_log10, JSON_FILE_BUSTEDS_MH, title_text, outfile_log10, True)
    #end if
    
#end method\

for BUSTEDSMH_File in BUSTEDS_MH_DIR_FILES:
    basename = os.path.basename(BUSTEDSMH_File).replace(".nex.BUSTEDS-MH.json", "")
    # Find BUSTEDS File
    BUSTEDS_File = os.path.join(BUSTEDS_DIR, basename + ".nex.BUSTEDS.json")
    print(BUSTEDSMH_File, BUSTEDS_File)
    
    main(BUSTEDSMH_File, BUSTEDS_File)

/Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS-MH/InfluenzaA.nex.BUSTEDS-MH.json /Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS/InfluenzaA.nex.BUSTEDS.json
# Reading: /Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS-MH/InfluenzaA.nex.BUSTEDS-MH.json
# Reading: /Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS/InfluenzaA.nex.BUSTEDS.json
# No ER data

/Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS-MH/COXI.nex.BUSTEDS-MH.json /Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS/COXI.nex.BUSTEDS.json
# Reading: /Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS-MH/COXI.nex.BUSTEDS-MH.json
# Reading: /Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS/COXI.nex.BUSTEDS.json
# No ER data

/Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS-MH/flavNS5.nex.BUSTEDS-MH.json /Users/user/Documents/BUSTEDS-MH/analysis/13-datasets/BUSTEDS/flavNS5.nex.BUSTEDS.json
# Reading: /Users/user/Docum

In [48]:
type(result)


pandas.core.frame.DataFrame